In [ ]:
import xml.etree.ElementTree as ET
import dateutil.parser
import math

In [ ]:
import haversine

class TrackPoint:
    def __init__(self, pt):
        self.lat = float(pt.attrib['lat'])
        self.lon = float(pt.attrib['lon'])
        self.ele = float(pt.find('{http://www.topografix.com/GPX/1/1}ele').text)
        self.date_time = pt.find('{http://www.topografix.com/GPX/1/1}time').text
        self.dt = dateutil.parser.parse(self.date_time)

    @property
    def coord(self):
        return (self.lon, self.lat)

    def to_sql(self, prev_pt):
        secs = 0.0
        dist = 0.0
        xdist = 0.0
        if prev_pt:
            secs = (self.dt - prev_pt.dt).total_seconds()
            dist = haversine.distance(self.coord, prev_pt.coord)
            delta_ele = self.ele - prev_pt.ele
            xdist = math.sqrt((dist * dist + delta_ele * delta_ele))
        return (self.date_time, self.lat, self.lon, self.ele, str(self.dt.date()), secs, dist, xdist)

    def __repr__(self):
            return f'{self.date_time}, ({self.lat}, {self.lon}), {self.ele}m'


In [ ]:
class RouteParser:
    def __init__(self, path):
        root = ET.parse(path).getroot()
        self.pts = [TrackPoint(pt) for pt in root.findall('.//{http://www.topografix.com/GPX/1/1}trkpt')]

In [ ]:
import itertools
import sqlite3

def pairwise(iterable):
    prev_item = None
    for current_item in iterable:
        yield (current_item, prev_item)
        prev_item = current_item

class TrackPointDbBuilder:
    def __init__(self, db_path='mwalks.sqlite'):
        self.db_path = db_path

    def create_db(self):
        self.connect()
        self.conn.execute('DROP TABLE IF EXISTS points')
        sql = '''CREATE TABLE points (
            datetime_text TEXT,
            lat REAL,
            lon REAL,
            ele REAL,
            dt DATE,
            elapsed_secs REAL,
            distance REAL,
            xdistance REAL
        )'''
        self.conn.execute(sql)
        self.disconnect()

    def put_points(self, pts):
        insert_points = [curr_pt.to_sql(prev_pt) for curr_pt, prev_pt in pairwise(pts)]    
        conn = self.connect()
        csr = conn.cursor()
        sql = 'INSERT INTO points VALUES (?, ?, ?, ?, ?, ?, ?, ?)'
        res = csr.executemany(sql, insert_points)
        csr.close()
        conn.commit() 
        self.disconnect()  

    def connect(self):
        self.conn = sqlite3.connect(self.db_path)
        return self.conn

    def disconnect(self):
        self.conn.close()

In [ ]:
builder = TrackPointDbBuilder()
builder.create_db()

In [ ]:
from glob import glob

for path in glob('routes/*.gpx'):
    route = RouteParser(path)
    builder.put_points(route.pts)